<b>Optimization in Python</b>

In this notebook we are going to discuss the 'mature' field of optimization. Finding the min, max, longest, shrotest, fastest, cheapest, most expensive, etc.... Optimizaiton problems almost always have a known solution. So the work consists of breaking down your specific problem and mapping it onto a pre-existing solution.

The first instance we are going to look at is called the <b>0/1 knapsack problem</b>. Basically, when you have <b><i>limited capactiy</i></b> and you want to <b><i>optimize the value</i></b>  of <b><i>items</i></b> you get within those limits. Examples include:

- finding the maximum value of items you can put in your bag until it fills up or gets too heavy
- finding the the best tasting foods you can eat on a 1,500 calorie diet 
- Spending the least amount of money on groceries but ensuring you get over 1500 calories per day

These kinds of problems are inherently exponential. That means if we try to solve them, we would have to find the powerset, aka we must test out every single combination of food items to see which combo best optimizes values... This is called using a 'brute force' algorithm. In general, exponential problems really suck to calculate. For example, if you have 100 food items you could eat on your diet, you would need to test out 2^100 possible combinations of food to find out which combination is best.

In [55]:
# What does 2^100 look like??
print(2**100)
print('In perspective, that is', ((2**100)/82000000/60/60/24/365), 'years @ 82 million calculations per second.')

1267650600228229401496703205376
In perspective, that is 490206546845505.8 years @ 82 million calculations per second.


100 possible food items is a reasonable number of available choices for most people.
A zillion years is not a reasonable amount of time to decide what to eat.

In [37]:
class Food(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.calories = w
    def getValue(self):
        return self.value
    def getCost(self):
        return self.calories
    def density(self):
        return self.getValue()/self.getCost()
    def __str__(self):
        return self.name + ': <' + str(self.value)\
                 + ', ' + str(self.calories) + '>'

In [38]:
def buildMenu(names, values, calories):
    """names, values, calories lists of same length.
       name a list of strings
       values and calories lists of numbers
       returns list of Foods"""
    menu = []
    for i in range(len(values)):
        menu.append(Food(names[i], values[i],
                          calories[i]))
    return menu

In [39]:
def greedy(items, maxCost, keyFunction):
    """Assumes items a list, maxCost >= 0,
         keyFunction maps elements of items to numbers"""
    itemsCopy = sorted(items, key = keyFunction,
                       reverse = True)
    result = []
    totalValue, totalCost = 0.0, 0.0
    for i in range(len(itemsCopy)):
        if (totalCost+itemsCopy[i].getCost()) <= maxCost:
            result.append(itemsCopy[i])
            totalCost += itemsCopy[i].getCost()
            totalValue += itemsCopy[i].getValue()
    return (result, totalValue)

In [40]:
def testGreedy(items, constraint, keyFunction):
    taken, val = greedy(items, constraint, keyFunction)
    print('Total value of items taken =', val)
    for item in taken:
        print('   ', item)

In [58]:
def testGreedys(foods, maxUnits):
    print('Use greedy by value to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.getValue)
    print('\nUse greedy by cost to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits,
               lambda x: 1/Food.getCost(x))
    print('\nUse greedy by density to allocate', maxUnits,
          'calories')
    testGreedy(foods, maxUnits, Food.density)

Now it is time to set our menu items as well as their associated values and weights.

In [56]:
names = ['wine', 'beer', 'pizza', 'burger', 'fries',
         'cola', 'apple', 'donut', 'cake']
values = [89,90,95,100,90,79,50,10,25]
calories = [123,154,258,354,365,150,95,195,200]

#### In order to help you better visualize the data, let's put our menu with calories and values into a pandas DataFrame.

In [57]:
import pandas as pd
d = {'food': names, 'value': values, 'calories': calories}
df = pd.DataFrame(data=d)
df

,food,value,calories
0,wine,89,123
1,beer,90,154
2,pizza,95,258
3,burger,100,354
4,fries,90,365
5,cola,79,150
6,apple,50,95
7,donut,10,195
8,cake,25,200


In [43]:
foods = buildMenu(names, values, calories)
testGreedys(foods, 1000)

Use greedy by value to allocate 1000 calories
Total value of items taken = 424.0
    burger: <100, 354>
    pizza: <95, 258>
    beer: <90, 154>
    wine: <89, 123>
    apple: <50, 95>

Use greedy by cost to allocate 1000 calories
Total value of items taken = 343.0
    apple: <50, 95>
    wine: <89, 123>
    cola: <79, 150>
    beer: <90, 154>
    donut: <10, 195>
    cake: <25, 200>

Use greedy by density to allocate 1000 calories
Total value of items taken = 428.0
    wine: <89, 123>
    beer: <90, 154>
    cola: <79, 150>
    apple: <50, 95>
    pizza: <95, 258>
    cake: <25, 200>


You can see in the testGreedys func we set three benchmarks (value, cost, & density) for deciding how to rank food items best-worst. After looking at these answers, we see that all three benchmarks gave us different 'optimal solutions'... Each of these are called 'locally optimal' solutions. Greedy algorithms can only return locally optimal solutions. With a greedy algorithm, there is no way to tell 100% what the absolute optimal solution is.